# core

> Basic musy building blocks

In [ ]:
#|default_exp core

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
from fastcore.all import *
from mingus.core import chords, notes, intervals

# Base Variables

These variables contain the basic orderings for Western music theory.

In [ ]:
#|export
BASE_NOTES = ["C", "D", "E", "F", "G", "A", "B"]
CHROMATIC_NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
ENHARMONIC_NOTES = ["C", "C#", "Db", "D", "D#", "Eb", "E", "Fb", "E#", "F", "F#", "Gb",
                    "G", "G#", "Ab", "A", "A#", "Bb", "B", "Cb", "B#"]
INTERVALS = ["1", "b2", "2", "b3", "3", "4", "#4", "5", "b6", "6", "b7", "7"]

# Note

The `Note` is the basic atomic unit in music. Combinations of notes will form chords and scales. Notes and chords together will form songs.

In [ ]:
#|export
class Note(BasicRepr):
    def __init__(self, note: str):
        # Transform note to uppercase
        note = note[0].upper() + note[1:]
        assert notes.is_valid_note(note), f"Note '{note}' is not valid"
        self.note = self.postprocess_note(notes.remove_redundant_accidentals(note))

    @staticmethod
    def postprocess_note(note: str):
        """ Get rid of unnecessary accidentals."""
        if note == "B#": note = "C"
        elif note == "E#": note = "F"
        elif note == "Cb": note = "B"
        elif note == "Fb": note = "E"
        elif note.endswith("##"):
            note = BASE_NOTES[BASE_NOTES.index(note[0])+1]
        elif note.endswith("bb"):
            note = BASE_NOTES[BASE_NOTES.index(note[0])-1]
        return str(note)
    
    def __str__(self): return self.note
    def __eq__(self, other): return str(self) == str(other)
    def __ne__(self, other): return not str(self) == str(other) 

In [ ]:
a_sharp = Note("A#")
a_sharp

Note(note='A#')

In [ ]:
assert str(a_sharp) == "A#"
a_sharp.note

'A#'

In [ ]:
c_sharp = Note("C#")
c_sharp

Note(note='C#')

In [ ]:
assert Note("E#") == "F"
assert Note("B#") == "C"
assert Note("C##") == "D"
assert Note("Fb") == "E"
assert Note("Abb") == "G"
assert Note("Bbb") == "A"
assert Note("Cb") == "B"
assert Note("C") == Note("C")
assert Note("E#") == Note("F")
assert Note("A#") != Note("B")

## Adding to Note

Adding semitones to a note will return a new note with n semitones added above the original note.

For example, adding 1 semitone to A# will return B.

In [ ]:
#|export
@patch
def __add__(self:Note, semitones: int):
    """Add n semitones to a note."""
    return Note(intervals.from_shorthand(str(self), INTERVALS[(semitones)%12]))

In [ ]:
for i in range(1, 13):
    print(a_sharp+i)

B
C
C#
D
D#
E
F
F#
G
G#
A
A#


In [ ]:
assert str(a_sharp+1) == "B"
assert str(a_sharp+11) == "A"
assert str(a_sharp+12) == "A#"
assert str(a_sharp+13) == "B"

## Subtracting from Note

Subtracting semitones from a note will return a new note with `n` semitones subtracted from the original note.

For example, subtracting 1 semitone from C will return B. subtracting 1 semitone from A# will return A.


In [ ]:
#|export
@patch
def __sub__(self:Note, semitones: int):
    return Note(intervals.from_shorthand(str(self), INTERVALS[(semitones)%12], False))

In [ ]:
for i in range(1, 13):
    print(a_sharp-i)

A
G#
G
F#
F
E
D#
D
C#
C
B
A#


In [ ]:
assert str(a_sharp-1) == "A"
assert str(a_sharp-11) == "B"
assert str(a_sharp-12) == "A#"
assert str(a_sharp-13) == "A"

## Augment

Augmenting adds a sharp to a note.

For example, augmenting A# will return B.

In [ ]:
#|export
@patch
def augment(self:Note):
    return Note(str(self) + "#" if str(self)[-1] != "b" else str(self)[:-1])

In [ ]:
assert str(a_sharp.augment()) == "B"
a_sharp.augment()

Note(note='B')

`augment` operations can be chained together.

In [ ]:
assert str(a_sharp.augment().augment().augment().augment()) == "D"
a_sharp.augment().augment().augment().augment()

Note(note='D')

## Diminish

Diminishing a note is the inverse of augmenting a note and removes a sharp from a note.

For example, diminishing A# will return A.

In [ ]:
#|export
@patch
def diminish(self:Note):
    return Note(str(self) + "b" if str(self)[-1] != "#" else str(self)[:-1])

In [ ]:
assert str(a_sharp.diminish()) == "A"
a_sharp.diminish()

Note(note='A')

As with `augment`, `diminish` operations can be chained together.

In [ ]:
assert str(a_sharp.diminish().diminish()) == "Ab"
a_sharp.diminish().diminish()

Note(note='Ab')

Lastly, `augment` and `diminish` can be chained together.

In [ ]:
assert str(a_sharp.diminish().augment()) == str(a_sharp)
a_sharp.diminish().augment()

Note(note='A#')

## Interval

Two notes can be combined to form an interval. `interval` returns the name of the interval between two notes.

In [ ]:
#|export
@patch
def interval(self:Note, other:Note, short=False):
    return intervals.determine(str(self), str(other), short)

The intervals between `A#` and `C#` is a minor third (b3).

In [ ]:
c_sharp = Note("C#")
interval = a_sharp.interval(c_sharp)
assert interval == "minor third"
interval

'minor third'

Shorthand can be retrieved from `interval` by setting `short=True`.

In [ ]:
short_interval = a_sharp.interval(c_sharp, short=True)
assert short_interval == "b3"
short_interval

'b3'

## Convert to Major or Minor

A note can be converted to its relative major or minor. How this is converted can be visualized on the circle of fifths.

<img src="https://upload.wikimedia.org/wikipedia/commons/3/33/Circle_of_fifths_deluxe_4.svg" width="40%" alt="Circle of Fifths">

For example, the relative minor of C is A. The relative major of C# is E.

`minor` converts an arbitrary note to its relative minor. This is done by subtracting 3 semitones from the note.

In [ ]:
#|export
@patch
def minor(self:Note): return self - 3

In [ ]:
c = Note("C")
assert str(c.minor()) == "A"
c.minor()

Note(note='A')

`major` converts an arbitrary note to its relative major. We add 3 semitones to the note.

In [ ]:
#|export
@patch
def major(self:Note): return self + 3

In [ ]:
assert str(c_sharp.major()) == "E"
c_sharp.major()

Note(note='E')

# Chord

In [ ]:
# TODO Implement Chord Class

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()